In [1]:
import torch
# model_silero_te, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models',
#                                                                   model='silero_te')
import re
import string
from nltk.tokenize import word_tokenize,TreebankWordTokenizer


In [8]:
import soundfile
# load model and processor
import os
# import librosa
import jiwer
import string
import csv
from autocorrect import Speller
spell = Speller()
from transformers import pipeline

# fix_spelling = pipeline("text2text-generation",model="oliverguhr/spelling-correction-english-base")


def get_gold_hyp_lists(gold_path,test_path,septype):
    def create_hyp_sentence_dictionary(filename):
        sentence_dict = {}
        with open(filename, 'r') as file:
            lines = file.readlines()
            for idx,line in enumerate(lines):
                line = line.strip()
                if line and len(line.split(septype, 1))==2:
                    # print(line.split(septype, 1))
                    sentence_id, sentence = line.split(septype,1)
                    # sentence_id=sentence_id+".wav" # for AMI
                    sentence_id = sentence_id.split(".", 1)[0]  # cv
                    sentence_dict[sentence_id] = sentence
        return sentence_dict

    def create_gold_sentence_dictionary(filename):
        sentence_dict = {}
        with open(filename, 'r') as file:
            for line in file:
                # parts = line.split(' ', 1) # Libri
                parts = line.split(',', 1) # AMI
                sentence_id, sentence = parts
                # print(sentence_id)
                # sentence_id = sentence_id.split("/", 1)[1]  # AMI
                sentence_id = sentence_id.split(".", 1)[0]  # cv
                # print(sentence_id)
                # print(sentence.strip())
                if sentence.strip()!='':
                    sentence_dict[sentence_id] = sentence.strip()
        return sentence_dict

    def read_csv_to_dict(file_path):
        result_dict = {}
        with open(file_path, 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            for row in csv_reader:
                key, value = row[0], row[1]
                key=key.split('/')[1]
                # print(key,value)
                # break
                result_dict[key] = value
        return result_dict

    gold_dict = create_gold_sentence_dictionary(gold_path)
    # gold_dict = read_csv_to_dict(gold_path)
    test_dict = create_hyp_sentence_dictionary(test_path)

    gold_list = []
    test_list = []
    for test_key, test_value in test_dict.items():
        # print(test_key)
        if test_key in gold_dict:
            gold = gold_dict[test_key]
            gold_list.append(gold)
            test_list.append(test_value)
    print(len(test_list))
    return gold_list,test_list

def add_space_before_punctuation(text):
    # List of punctuation marks
    punctuation_marks = string.punctuation

    # Iterate over each character in the text
    modified_text = ''
    for char in text:
        if char in punctuation_marks:
            modified_text += ' ' + char
        else:
            modified_text += char

    return modified_text

def add_space_before_punctuation(text):
    # List of punctuation marks
    punctuation_marks = string.punctuation

    # Iterate over each character in the text
    modified_text = ''
    for char in text:
        if char in punctuation_marks:
            modified_text += ' ' + char
        else:
            modified_text += char

    return modified_text

def add_space(reference_list, hypothesis_list):
    reference = []
    hypothesis = []
    for ref, hyp in zip(reference_list, hypothesis_list):
        # Preprocess sentences to keep punctuation as separate words
        reference.append(add_space_before_punctuation(ref))
        hypothesis.append(add_space_before_punctuation(hyp))
        # print(reference,hypothesis)
        # Calculate WER
       
    #     wer = jiwer.wer(reference, hypothesis)
    #     # print(wer)
    #     wer_scores.append(wer)
    # wer_mean=sum(wer_scores) / len(wer_scores)
    
    return reference,hypothesis

def get_detail_wer(gold_list,test_list):
    out = jiwer.process_words(
            gold_list,
                test_list,
            )
    print("wer: ",out.wer*100)
    print("hits: ",out.hits)
    print("substitutions: ",out.substitutions)
    print("insertions: ",out.insertions)
    print("deletions: ",out.deletions)
    print("total words: ",out.hits+out.substitutions+out.deletions)

# def get_t_p_wer(substitutions, insertions,deletions,total_words_punc, P_WER):
#     t_p_wer=(substitutions+insertions+deletions)/total_words_punc
#     print("Topline of P-WER is ",t_p_wer*100)
#     print("Relative PER is ", P_WER-t_p_wer*100)

def get_c_p_er_fr_wer(wer,C_WER,P_WER,clean=True):
    if clean:
        total_words=50082
        total_words_punt=57312
        cased_words=3849
        p_num = 9709
    else:
        total_words=48488
        total_words_punt=55896
        cased_words=4231
        p_num = 9408
    error_words = wer*total_words
    error_c = C_WER*total_words
    cer=(error_c-error_words)/cased_words
    error_p = P_WER*total_words_punt
    per=(error_p-error_words)/p_num
    # print("Topline of P-WER is ",t_p_wer)
    print("Relative CER is ", cer)
    print("Relative PER is ", per)

def get_c_p_er_fr_wer_ami(wer,C_WER,P_WER):
    # if clean:
    total_words=94006
    total_words_punt=110266
    cased_words=7626
    p_num = 16260
    error_words = wer*total_words
    error_c = C_WER*total_words
    cer=(error_c-error_words)/cased_words
    error_p = P_WER*total_words_punt
    per=(error_p-error_words)/p_num
    print("Relative CER is ", cer)
    print("Relative PER is ", per)


def get_c_p_er_fr_wer_cv(wer, C_WER, P_WER):
    # if clean:
    total_words = 11244
    total_words_punt = 13146
    cased_words = 1882
    p_num = 1902
    error_words = wer * total_words
    error_c = C_WER * total_words
    cer = (error_c - error_words) / cased_words
    error_p = P_WER * total_words_punt
    per = (error_p - error_words) / p_num
    print("Relative CER is ", cer)
    print("Relative PER is ", per)


def get_cased_word(text_list):
    capitalized_word_count = 0

    # Iterate through the list of text
    for text in text_list:
        # Split the text into words
        words = text.split()
        
        # Check each word in the text
        for word in words:
            # Check if the word is capitalized (first character is uppercase)
            if word.isalpha() and word[0].isupper():
                capitalized_word_count += 1
    print("Cased words number is ", capitalized_word_count)

In [7]:
gold_list, test_list = get_gold_hyp_lists(
    "/Users/ccui/Desktop/POC/dictation/CommonVoice/cv-en_test-2k_len4-10.txt",
    "cv-40.txt",
    ",",
)
get_cased_word(gold_list)
gold_list, test_list = add_space(gold_list, test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
gold_rc = jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp = jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
# gold_rp,test_rp= add_space(gold_rp,test_rp)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp = jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

1537
Cased words number is  1882
CP-WER
wer:  29.27886809675947
hits:  9770
substitutions:  2737
insertions:  473
deletions:  639
total words:  13146
P-WER
wer:  27.73467214361783
hits:  9976
substitutions:  2528
insertions:  476
deletions:  642
total words:  13146
C-WER
wer:  29.722518676627534
hits:  8347
substitutions:  2505
insertions:  445
deletions:  392
total words:  11244
wer:  27.810387762362147
hits:  8564
substitutions:  2286
insertions:  447
deletions:  394
total words:  11244


In [ ]:
get_c_p_er_fr_wer_cv(
    wer=27.810387762362147,
    C_WER=5.345309381237525,
    P_WER=8.016185858303974,
)

In [24]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/POC/dictation/books-transcription/test-clean-rich-book.txt","clean.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
# gold_rp,test_rp= add_space(gold_rp,test_rp)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

2491
CP-WER
wer:  11.602190672201486
hits:  51058
substitutions:  3385
insertions:  376
deletions:  2891
total words:  57334
P-WER
wer:  9.585935047266894
hits:  52211
substitutions:  2235
insertions:  373
deletions:  2888
total words:  57334
C-WER
wer:  4.770459081836327
hits:  47824
substitutions:  2138
insertions:  114
deletions:  138
total words:  50100
wer:  2.3552894211576847
hits:  49034
substitutions:  928
insertions:  114
deletions:  138
total words:  50100


In [31]:
get_c_p_er_fr_wer(
    wer=3.255489021956088,
    C_WER=5.345309381237525,
    P_WER=8.016185858303974,
    clean=True,
)

Relative CER is  27.192097488577012
Relative PER is  30.526546782728655


In [5]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/dictation/books-transcription/test-other-rich-book.txt","other.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
# print("Total WER:", total_wer*100)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
# gold_rp,test_rp= add_space(gold_rp,test_rp)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
print("WER")
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

2731
CP-WER
wer:  14.944296418161334
hits:  48012
substitutions:  4786
insertions:  448
deletions:  3123
total words:  55921
P-WER
wer:  12.742976699272187
hits:  49241
substitutions:  3559
insertions:  446
deletions:  3121
total words:  55921
C-WER
wer:  8.12460057311316
hits:  44799
substitutions:  3388
insertions:  233
deletions:  320
total words:  48507
WER
wer:  5.494052404807554
hits:  46075
substitutions:  2112
insertions:  233
deletions:  320
total words:  48507


In [30]:
get_c_p_er_fr_wer(
    wer=7.6132281904132775,
    C_WER=9.888113620412039,
    P_WER=12.22874740385304,
    clean=False,
)

Relative CER is  26.07058490422593
Relative PER is  33.41707657196115


In [67]:
gold_list, test_list = get_gold_hyp_lists(
    "/Users/ccui/Desktop/POC/dictation/vasr/calculate_wer/ami_test_gold_process.txt",
    "/Users/ccui/Desktop/POC/dictation/whisper/whisper-small-en-ami-rtf.txt",
    " ",
)
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
# print("Total WER:", total_wer*100)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

9500
CP-WER
wer:  30.41117496691383
hits:  82065
substitutions:  13176
insertions:  5296
deletions:  15077
total words:  110318
P-WER
wer:  27.55669972262006
hits:  85236
substitutions:  9983
insertions:  5318
deletions:  15099
total words:  110318
C-WER
wer:  26.026479502312966
hits:  71772
substitutions:  9812
insertions:  2211
deletions:  12451
total words:  94035
wer:  22.552241186792152
hits:  75047
substitutions:  6529
insertions:  2219
deletions:  12459
total words:  94035


In [68]:
get_c_p_er_fr_wer_ami(
    wer=22.552241186792152, C_WER=26.026479502312966, P_WER=27.55669972262006
)

Relative CER is  42.827071477688115
Relative PER is  56.48961049254862
